**Here is code showcasing how Geopy, a Python library, can be employed to extract the latitude and longitude coordinates for addresses provided in a dataframe.**

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=fb59540e5ca14308e544580e5f22e14bc8364bc9463d7251582bf5a59cafbcb9
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Create a Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
#Read json file in spark
df = spark.read.json("/content/drive/MyDrive/Project_v2/data_1.jsonl")

In [5]:
#Print the schema of the dataset
df.printSchema()

root
 |-- __twarc: struct (nullable = true)
 |    |-- retrieved_at: string (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attachments: struct (nullable = true)
 |    |    |    |-- media_keys: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- poll_ids: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- context_annotations: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- domain: struct (nullable = true)
 |    |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- entity: str

In [6]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd

In [8]:
#Select "id" and "location" columns from "users" within the "includes" column
users_location = df.select("includes.users.id","includes.users.location")
users_location.printSchema()

root
 |-- id: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- location: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [9]:
#display the dataframe
users_location.show()

+--------------------+--------------------+
|                  id|            location|
+--------------------+--------------------+
|[2359337692, 1378...|[Florida, USA, Wi...|
|[2535367145, 3510...|[null, Western Ma...|
|[862197469, 21677...|[Delaware, USA, V...|
|[1301306322, 2167...|[null, Vermont, L...|
|[1034916304296132...|[Los Angeles, Ver...|
|[26925474, 260971...|[Estelline, SD, W...|
|[1117602100823908...|[London, England,...|
|[847782908, 13431...|[London, Ontario,...|
|[92808302, 130557...|[London, Washingt...|
|[1472486317, 1392...|[En un lugar del ...|
|[2704573500, 1349...|[United States, N...|
|[278889835, 21677...|[null, Vermont, M...|
|[9505213206802104...|[Davis, CA, null,...|
|[258741909, 32954...|[null, GA6, Unite...|
|[64042956, 134949...|[Warrenton, MO, N...|
|[190666108, 28581...|[Bozeman, MT, nul...|
|[1452332784, 2167...|[West Texas, Verm...|
|[133793696, 19344...|[null, Seattle, W...|
|[8299581051821096...|[null, Vermont, p...|
|[20919324, 216776...|[null, Ver

In [10]:
#Explode the json data so that each row has a single element and not an array element
from pyspark.sql.functions import arrays_zip, explode

new_df = users_location.select("id",explode(arrays_zip("id", "location")))
new_df.show()

+--------------------+--------------------+
|                  id|                 col|
+--------------------+--------------------+
|[2359337692, 1378...|{2359337692, Flor...|
|[2359337692, 1378...|{1378022598752223...|
|[2359337692, 1378...|{627019754, Georgia}|
|[2359337692, 1378...|{216776631, Vermont}|
|[2359337692, 1378...|{494287469, Occup...|
|[2359337692, 1378...|  {2155641480, null}|
|[2359337692, 1378...|   {939296060, null}|
|[2359337692, 1378...|   {752952668, null}|
|[2359337692, 1378...|{67358777, Homes,...|
|[2359337692, 1378...|{8407332989890969...|
|[2359337692, 1378...|{1201902968355905...|
|[2359337692, 1378...|{1428498179369193...|
|[2359337692, 1378...|  {2539762034, null}|
|[2359337692, 1378...|{1352735428726632...|
|[2359337692, 1378...|{13340732, London...|
|[2359337692, 1378...|{7373708034113904...|
|[2359337692, 1378...|   {358530210, null}|
|[2359337692, 1378...|   {192944972, null}|
|[2359337692, 1378...|{8373319925806202...|
|[2359337692, 1378...|  {2181308

In [11]:
#Unzipping array elements from "col" column to seperate columns

new_df_2 = new_df.select("col.id","col.location")
new_df_2.show()

+-------------------+--------------------+
|                 id|            location|
+-------------------+--------------------+
|         2359337692|        Florida, USA|
|1378022598752223235|   Wichita Falls, TX|
|          627019754|             Georgia|
|          216776631|             Vermont|
|          494287469|Occupied Lenapeho...|
|         2155641480|                null|
|          939296060|                null|
|          752952668|                null|
|           67358777|Homes, screens, a...|
| 840733298989096960|                  DC|
|1201902968355905543|      Washington, DC|
|1428498179369193476|                null|
|         2539762034|                null|
|1352735428726632449|        Florida, USA|
|           13340732|London, England, ...|
| 737370803411390464|          Layton, UT|
|          358530210|                null|
|          192944972|                null|
| 837331992580620288|Port Harcourt, Ni...|
|         2181308226|                null|
+----------

In [12]:
#convert pyspark dataframe to Pandas dataframe
location_data = new_df_2.toPandas()

In [ ]:
#delete location with null values
location_data = location_data.dropna()

In [ ]:
#Print the total number of datapoints retrieved
location_data.shape

(634359, 2)

In [ ]:
#Print the initial 5 rows of the dataframe 
location_data.head()

,id,location
0,2359337692,"Florida, USA"
1,1378022598752223235,"Wichita Falls, TX"
2,627019754,Georgia
3,216776631,Vermont
4,494287469,Occupied Lenapehoking land


In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [ ]:
# Function to handle timeout errors
def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [ ]:
# Initialize geolocator
geolocator = Nominatim(user_agent="myGeocoder")

**This is where the actual converstion from addresses to coordinates take place**

In [ ]:
# Loop through dataframe and geocode addresses
for i, row in location_data.iterrows():
    address = row['location']
    try:
        location = do_geocode(address)
        if location is not None:
            location_data.at[i, 'latitude'] = location.latitude
            location_data.at[i, 'longitude'] = location.longitude
        else:
            print(f"Geocoding failed for address: {address}")
    except Exception as e:
        print(f"Error geocoding address '{address}': {e}")

Geocoding failed for address: Occupied Lenapehoking land
Geocoding failed for address: Homes, screens, and stores
Geocoding failed for address: dιrтyjerѕey | aтlanтaѕтan
Geocoding failed for address: Hawaii Occupied Kingdom(HOK)


Geocoding failed for address: Carolina In My Mind
Geocoding failed for address: Sydney. Cammerraygal Lands
Geocoding failed for address: Kgn, Jamaica/Jamaica Plain, MA
Geocoding failed for address: York, our Green & Septic isle
Geocoding failed for address: New York City, Philly raised
Geocoding failed for address: Werescrewed
Geocoding failed for address: SE Armpit, GA
Geocoding failed for address: https://ko-fi.com/critfacts
Geocoding failed for address: Playing... Something 
Geocoding failed for address: United Corporations of America
Geocoding failed for address: 🪐
Geocoding failed for address: Los Angeles, CA/Seattle, WA
Geocoding failed for address: New York City, Philly raised
Geocoding failed for address: probably on Earth
Geocoding failed for address: Hollow Bastion, Florida x PA
Geocoding failed for address: Join free newsletter here...
Geocoding failed for address: Burlington, Saugeen Shores, ON
Geocoding failed for address: Ennui-sur-Blasé
Geocoding failed for address: Hunt

Error geocoding address 'Phoenixville P.A.': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Phoenixville+P.A.&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Geocoding failed for address: Lenape land (Poughkeepsie, NY)
Geocoding failed for address: Retweet With Comments
Geocoding failed for address: I Am Literally In Your Palm
Geocoding failed for address: Joaquín Gress.ᥫ᭡
Geocoding failed for address: North San Diego & SW Riverside, Assembly District 75
Geocoding failed for address: World Wide


Geocoding failed for address: The Idiot's Lantern
Geocoding failed for address: Loli Bitch Island
Geocoding failed for address: ............
Geocoding failed for address: Cooltopia 🐱
Geocoding failed for address: Wiyot Territory (CA)
Geocoding failed for address: World of Women Galaxy
Geocoding failed for address: 🇺🇸🇮🇹
Geocoding failed for address: Now live in Portland, Oregon 
Geocoding failed for address: she/her•21
Geocoding failed for address: ireland she/her
Geocoding failed for address: Indianpolis IN
Geocoding failed for address: Somewhere Some place
Geocoding failed for address: everywhere and nowhere
Geocoding failed for address: #EyesWideOpen #KnowYourChemistry
Geocoding failed for address: New York City, Philly raised
Geocoding failed for address: Kaurna Country
Geocoding failed for address: Lenapehoking/landback.org


Error geocoding address 'Maryland's 8th District': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Maryland%27s+8th+District&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Geocoding failed for address: Dublin/cairo
Geocoding failed for address: sunny - she/her - 26


...

Error geocoding address 'South West, England': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=South+West%2C+England&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Geocoding failed for address: IIT Gauhati, Guwahati
Geocoding failed for address: Mount Weather Quarantine
Geocoding failed for address: Past the black stump...
Geocoding failed for address: Europe, occasionally UK.
Geocoding failed for address: thokkalodhi
Geocoding failed for address: MNG offices, Ndemi Rd Kilimani
Geocoding failed for address: American held captive in UK 
Geocoding failed for address: MollySkyeBrown


Error geocoding address 'México - L.A - Madrid - London': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=M%C3%A9xico+-+L.A+-+Madrid+-+London&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Geocoding failed for address: coolandfunnytshirts@gmail.com
Geocoding failed for address: GlennGreenwald@gmail.com
Geocoding failed for address: Ohlone land (Richmond, CA) 
Geocoding failed for address: http://twitch.tv/smgxprincess
Geocoding failed for address: In your feelings
Geocoding failed for address: staceyabrams.com
Geocoding failed for address: Real Swag
Geocoding failed for address: 26 • she/her
Geocoding failed for address: 🇪🇹
Geocoding failed for address: Pittsburgh, PA via Buffalo, NY
Geocoding failed for address: 🌌
Geocoding failed for address: i:azuukichan/h:JL_T4N she/they
Geocoding failed for address: she/her; 23; white; bi; ADHD
Geocoding failed for address: Mount Weather Quarantine
Geocoding failed for address: COL/USA | 김려욱
Geocoding failed for address: 핑크령구나라


Geocoding failed for address: she/her • 25
Geocoding failed for address: living life the way I want 🖤
Geocoding failed for address: Somebody's Hospital
Geocoding failed for address: Stolen Land, Toronto 


KeyboardInterrupt: ignored

In [ ]:
#delete rows where null values were returned
location_data = location_data.dropna()

In [ ]:
location_data.head()

,id,location,latitude,longitude
0,2359337692,"Florida, USA",27.756767,-81.463983
1,1378022598752223235,"Wichita Falls, TX",33.900457,-98.502078
2,627019754,Georgia,32.329381,-83.113737
3,216776631,Vermont,44.599072,-72.500261
9,840733298989096960,DC,38.895037,-77.036543


In [ ]:
location_data.to_csv("location.csv")

In [ ]:
location_data.shape

(25445, 4)

**References**



*   Clean Location Data: https://towardsdatascience.com/cleaning-location-data-with-geopy-51613bcd3c89 


